In [1]:
%load_ext autoreload
%autoreload 2

In [9]:
import os
import json
from pprint import pprint

from src.supabase_client import get_user, list_users
from tests.common import run_new_training_week_process_wrapped, run_mid_week_update_process_wrapped


# Test: One Offs

In [ ]:
mock_user = get_user(105359460)
response = run_new_training_week_process_wrapped(mock_user)
print(response)

In [ ]:
mock_user = get_user(os.environ["JAMIES_ATHLETE_ID"])
response = run_mid_week_update_process_wrapped(mock_user)
print(response)

# Evals: New Training Week

In [ ]:
path = 'tests/artifacts/new_training_week_errors'
files = os.listdir(path)

for file in sorted(files):
    print(f"Loading {file}")
    with open(os.path.join(path, file), "r") as f:
        results = json.load(f)
    print(sum(result['error'] for result in results) / len(results))

In [ ]:
for result in results:
    print(result['user']['email'])
    print("target:", result['recommended_weekly_mileage'])
    print("actual:", result['generated_weekly_mileage'])
    print("error:", result['error'])
    print(result['planning'])
    print("." * 100)

# Manual: New Training Week

In [ ]:
from freezegun import freeze_time
from src.activities import get_activities_df, get_day_of_week_summaries, get_weekly_summaries
from src.auth_manager import get_strava_client
from src.constants import COACH_ROLE
from src.new_training_week import generate_training_week_with_coaching
from src.supabase_client import list_users

@freeze_time("2024-09-01")
def gen_training_week_wrapper(sysmsg_base, strava_client):
    activities_df = get_activities_df(strava_client)
    day_of_week_summaries = get_day_of_week_summaries(activities_df)
    weekly_summaries = get_weekly_summaries(activities_df)
    return generate_training_week_with_coaching(
        sysmsg_base=sysmsg_base,
        weekly_summaries=weekly_summaries,
        day_of_week_summaries=day_of_week_summaries,
    )

# "voynow99@gmail.com", "jaredpalek8@gmail.com", "laurenbrooks01@gmail.com", "raytrip.curtis@gmail.com"
for user in list_users():
    if user.email == "bconnolly04@gmail.com":
        print(user.email)
        sysmsg_base = f"{COACH_ROLE}\nYour client has included the following preferences: {user.preferences}\n"
        strava_client = get_strava_client(user.athlete_id)
        training_week_with_coaching = gen_training_week_wrapper(sysmsg_base, strava_client)
        print(training_week_with_coaching)
        break

In [ ]:
print(training_week_with_coaching.total_weekly_mileage)
print(training_week_with_coaching.weekly_mileage_target)
training_week_with_coaching.training_week

In [ ]:
from src.supabase_client import upsert_training_week_with_coaching

upsert_training_week_with_coaching(user.athlete_id, training_week_with_coaching)

In [ ]:
from src.email_manager import new_training_week_to_html, send_email
from src.lambda_function import get_athlete_full_name


send_email(
    to={"email": user.email, "name": get_athlete_full_name(strava_client)},
    subject="🏃‍♂️🎯 Just Fixed Your Jawn 🫡",
    html_content=new_training_week_to_html(training_week_with_coaching),
)